# Unitary ansatz entering the VQE

The final energy output of a VQE calculation will crucially depend on the ansatz/form of the parameterized unitary $\hat U(\boldsymbol{\theta})$ employed in state preparation. Here we review two popular approaches, the unitary coupled cluster and qubit coupled cluster methodologies, and benchmark them for energy calculations of small molecules.

In [1]:
import numpy as np
import tequila as tq
from utility import *
threshold = 1e-6 #Cutoff for UCC MP2 amplitudes and QCC ranking gradients

## Unitary Coupled Cluster (UCC)

The UCC ansatz is obtained by 'unitarizing' the traditional coupled cluster ansatz,
$$ e^{\hat T} \rightarrow e^{\hat T - \hat T^\dagger} \equiv \hat U_{\text{UCC}}$$


Due to non-commutativity of terms in $\hat T - \hat T^\dagger$, the UCC ansatz does not have a straightforward decomposition in terms of circuit primitives implementable on the quantum computer. Therefore, to obtain a form which can be compiled, we employ the Trotter approximation. The accuracy of the circuit ansatz relative to the exact UCC operator will be dependent on how many Trotter steps are employed. The number of Trotter steps is commonly set to its minimal value of one to avoid excessive circuit depth.

In [2]:
trotter_steps = 1

### H4 in STO-3G basis

Below is a sample VQE simulation using the UCCSD ansatz compiled using a single trotter step for H$_4$ in minimal basis at $R=2.5$ (Angstrom). For comparison, we can run FCI to obtain the true ground state energy.


In [3]:
xyz_data = get_molecular_data('h4', geometry=2.5, xyz_format=True)
basis='sto-3g'

h4 = tq.quantumchemistry.Molecule(geometry=xyz_data, basis_set=basis)

print('Number of spin-orbitals (qubits): {} \n'.format(2*h4.n_orbitals))

E_FCI = h4.compute_energy(method='fci')

print('FCI energy: {}'.format(E_FCI))

Number of spin-orbitals (qubits): 8 

FCI energy: -1.8731741642694442


The UCCSD VQE optimization is of the form
$$E = \min_{\boldsymbol{\theta}} \langle \text{HF} | \hat U_{\text{UCC}}^\dagger(\boldsymbol{\theta}) \hat H  \hat U_{\text{UCC}} (\boldsymbol{\theta}) | \text{HF} \rangle $$
To expedite the optimization process, we can set the initial guess of the amplitudes to zero, i.e. the optimization will begin at the Hartree Fock state. This heuristic is best suited for when Hartree Fock is believed to be in qualitative agreement with the true ground state. To further alleviate quantum resources, we can estimate the amplitudes using classical electronic structure methods (here, MP2 perturbation theory), and only include the unitaries with non-zero estimated amplitudes.

In [4]:
H = h4.make_hamiltonian()

print("\nHamiltonian has {} terms\n".format(len(H)))

U_UCCSD = h4.make_uccsd_ansatz(initial_amplitudes='MP2',threshold=threshold, trotter_steps=trotter_steps)

E = tq.ExpectationValue(H=H, U=U_UCCSD)

print('\nNumber of UCCSD amplitudes: {} \n'.format(len(E.extract_variables())))

print('\nStarting optimization:\n')

result = tq.minimize(objective=E, method="BFGS", initial_values={k:0.0 for k in E.extract_variables()}, tol=1e-6)

print('\nObtained UCCSD energy: {}'.format(result.energy))


Hamiltonian has 93 terms


Number of UCCSD amplitudes: 8 


Starting optimization:

Optimizer: <class 'tequila.optimizers.optimizer_scipy.OptimizerSciPy'> 
backend         : qulacs
samples         : None
save_history    : True
noise           : None

Method          : BFGS
Objective       : 1 expectationvalues
gradient        : 512 expectationvalues

active variables : 8

E=-1.40085345  angles= {(3, 1, 2, 0): 0.0, (3, 0, 3, 0): 0.0, (2, 0, 3, 1): 0.0, (2, 1, 2, 1): 0.0, (3, 1, 3, 1): 0.0, (3, 0, 2, 1): 0.0, (2, 0, 2, 0): 0.0, (2, 1, 3, 0): 0.0}  samples= None
E=-1.60312836  angles= {(3, 1, 2, 0): -0.4363546669483185, (3, 0, 3, 0): -0.2504887580871582, (2, 0, 3, 1): -0.4363546669483185, (2, 1, 2, 1): -0.25312143564224243, (3, 1, 3, 1): -0.3228883743286133, (3, 0, 2, 1): -0.12876605987548828, (2, 0, 2, 0): -0.30516868829727173, (2, 1, 3, 0): -0.12876605987548828}  samples= None
E=-1.82625120  angles= {(3, 1, 2, 0): -0.5609649449670754, (3, 0, 3, 0): -0.36704545557063445, (2, 0, 3, 1): -

We see that the converged UCCSD energy is in exact agreement with the FCI energy, as expected for a $2$-electron system. 

### LiH in 6-31G basis

Now let us try a larger problem of LiH in 6-31G basis. However, we will restrict the active space. The unrestricted problem leads to a $14$-qubit Hamiltonian, and $34$ UCCSD amplitudes to optimize even after neglecting the zero MP2 amplitudes. Therefore, we will remove some orbital degrees of freedom which are less important in accurately describing the electronic structure. By freezing all orbitals other than $0b_1$, $1b_1$, $2a_1$, and $3a_1$, we reduce the problem to an $8$-qubit Hamiltonian with $8$ UCCSD variational amplitudes.

In [6]:
xyz_data = get_molecular_data('lih', geometry=1, xyz_format=True)

basis = '6-31g'
active = {'B1':[0,1], 'A1':[2,3]}
lih = tq.quantumchemistry.Molecule(geometry=xyz_data, basis_set = basis, active_orbitals = active)

print('Number of spin-orbitals (qubits): {} \n'.format(2*lih.n_orbitals))

E_FCI = lih.compute_energy(method='fci')

print('FCI energy: {}'.format(E_FCI))

There are known issues with some psi4 methods and frozen virtual orbitals. Proceed with fingers crossed for hf.
Number of spin-orbitals (qubits): 8 

There are known issues with some psi4 methods and frozen virtual orbitals. Proceed with fingers crossed for fci.


SystemError: <built-in method detci of PyCapsule object at 0x7fd0af9191b0> returned NULL without setting an error

We will then run the UCCSD VQE simulation (***warning: tq.minimize will take several minutes - 1 hour + to converge for a VQE instance of this size.*** Smaller active spaces can be employed to lower VQE simulation runtimes).

In [7]:
H = lih.make_hamiltonian()

print("\nHamiltonian has {} terms\n".format(len(H)))

U_UCCSD = lih.make_uccsd_ansatz(initial_amplitudes='MP2',threshold=threshold, trotter_steps=trotter_steps)

E = tq.ExpectationValue(H=H, U=U_UCCSD)

print('\nNumber of UCCSD amplitudes: {} \n'.format(len(E.extract_variables())))

print('\nStarting optimization:\n')

result = tq.minimize(objective=E, method="BFGS", initial_values={k:0.0 for k in E.extract_variables()}, tol=1e-4)

print('\nObtained UCCSD energy: {}'.format(result.energy))


Hamiltonian has 185 terms



Exception: Received no targets upon initialization

We obtain a UCCSD energy within chemical accuracy (< 1.6 miliHartree from FCI energy) for the H$_2$O molecule near equilibrium geometry. As the bond distance approaches the dissociation limit, the energy deviation from FCI is typically expected to increase as electronic correlations increase during covalent bond-breaking. Furthermore, as HF becomes energetically more distant from the FCI wavefunction, the initial guess of all amplitudes being zero may lead to a local minimum. One can then instead initialize the amplitudes using random guesses, and repeat for $n$ samples to attempt to find the global minimum. To facilitate this, one can run the commented lines below. This will execute $n$ instances of tq.minimize for uniformly random initial parameters in interval $[0, 2\pi]$ and return the lowest optimization result.

In [8]:
n=10
result = minimize_E_random_guesses(objective=E, method='BFGS', tol=1e-4, n=n)

Optimizer: <class 'tequila.optimizers.optimizer_scipy.OptimizerSciPy'> 
backend         : qulacs
samples         : None
save_history    : True
noise           : None

Method          : BFGS
Objective       : 1 expectationvalues
gradient        : 512 expectationvalues

active variables : 8

E=-1.03269142  angles= {(3, 1, 2, 0): 0.6642716670111224, (3, 0, 3, 0): 4.81582964502689, (2, 0, 3, 1): 4.273390919921093, (2, 1, 2, 1): 1.3752290291352494, (3, 1, 3, 1): 0.7851086687725245, (3, 0, 2, 1): 5.291663632998772, (2, 0, 2, 0): 3.526979782002918, (2, 1, 3, 0): 2.006678072178764}  samples= None
E=-1.12988008  angles= {(3, 1, 2, 0): 0.5192932033613726, (3, 0, 3, 0): 4.954520831931797, (2, 0, 3, 1): 4.199354679343395, (2, 1, 2, 1): 1.346386818365596, (3, 1, 3, 1): 0.6525302958172022, (3, 0, 2, 1): 5.384472952494927, (2, 0, 2, 0): 3.41127450050687, (2, 1, 3, 0): 1.9965786499921507}  samples= None
E=-1.53597910  angles= {(3, 1, 2, 0): -0.06062065123762639, (3, 0, 3, 0): 5.509285579551426, (2, 0,

KeyboardInterrupt: 

## Qubit Coupled Cluster (QCC)

In contrast to UCC, the QCC methodology makes no direct reference to fermionic algebra and seeks to construct an efficient ansatz directly in qubit-space by finding multi-qubit Pauli strings (entanglers) which lower energy. This is done through an energy-lowering heuristic employing the energy gradient with respect to a Pauli strings variational amplitude. As opposed to UCCSD, the circuit depth and number of parameter is chosen to meet hardware limitations, i.e. one must choose how many exponentiated Pauli strings will be entering the QCC ansatz.

### H4 in STO-3G basis

Below we perform the entangler screening protocol for H4 in minimal basis, and obtain one grouping of entanglers with non-zero energy gradient. We then select one of them to be used in the QCC VQE simulation.

In [9]:
xyz_data = get_molecular_data('h4', geometry=2.5, xyz_format=True)
basis='sto-3g'

h4 = tq.quantumchemistry.Molecule(geometry=xyz_data, basis_set='sto-3g')

hf_reference = hf_occ(2*h4.n_orbitals, h4.n_electrons)

H = h4.make_hamiltonian()

print("\nHamiltonian has {} terms\n".format(len(H)))

#Define number of entanglers to enter ansatz
n_ents = 1

#Rank entanglers using energy gradient criterion
ranked_entangler_groupings = generate_QCC_gradient_groupings(H.to_openfermion(), 
                                                             2*h4.n_orbitals, 
                                                             hf_reference, 
                                                             cutoff=threshold)

print('Grouping gradient magnitudes (Grouping : Gradient magnitude):')
for i in range(len(ranked_entangler_groupings)):
    print('{} : {}'.format(i+1,ranked_entangler_groupings[i][1]))


entanglers = get_QCC_entanglers(ranked_entangler_groupings, n_ents, 2*h4.n_orbitals)

print('\nSelected entanglers:')
for ent in entanglers:
    print(ent)




Hamiltonian has 93 terms

Grouping gradient magnitudes (Grouping : Gradient magnitude):
1 : 0.1614
2 : 0.1567
3 : 0.1567
4 : 0.1526
5 : 0.1266
6 : 0.1259
7 : 0.1259
8 : 0.1252
9 : 0.0308
10 : 0.0308

Selected entanglers:
1.0 [X2 Y3 X6 X7]


Once the QCC ranking procedure has been ran, we can simulate the QCC VQE optimization with the generated entanglers. The VQE optimization for the QCC ansatz is of the form
$$E = \min_{\boldsymbol{\Omega}, \boldsymbol{\tau}} \langle \boldsymbol{\Omega} | U_{\text{ENT}}^\dagger (\boldsymbol{\tau}) \hat H  U_{\text{ENT}} (\boldsymbol{\tau}) | \boldsymbol{\Omega} \rangle $$
where $\boldsymbol{\Omega}$ denote collective Euler angles parameterizing single-qubit rotations, and $\boldsymbol{\tau}$ are entangler amplitudes. 

In [10]:
#Mean-field part of U (Omega):    
U_MF = construct_QMF_ansatz(n_qubits = 2*h4.n_orbitals)
#Entangling part of U:
U_ENT = construct_QCC_ansatz(entanglers)

U_QCC = U_MF + U_ENT

E = tq.ExpectationValue(H=H, U=U_QCC)

initial_vals = init_qcc_params(hf_reference, E.extract_variables())

#Minimize wrt the entangler amplitude and MF angles:
result = tq.minimize(objective=E, method="BFGS", initial_values=initial_vals, tol=1.e-6)

print('\nObtained QCC energy ({} entanglers): {}'.format(len(entanglers), result.energy))

Optimizer: <class 'tequila.optimizers.optimizer_scipy.OptimizerSciPy'> 
backend         : qulacs
samples         : None
save_history    : True
noise           : None

Method          : BFGS
Objective       : 1 expectationvalues
gradient        : 34 expectationvalues

active variables : 17

E=-1.40085345  angles= {beta_0: 3.141592653589793, gamma_0: 0.0, beta_1: 3.141592653589793, gamma_1: 0.0, beta_2: 3.141592653589793, gamma_2: 0.0, beta_3: 3.141592653589793, gamma_3: 0.0, beta_4: 0.0, gamma_4: 0.0, beta_5: 0.0, gamma_5: 0.0, beta_6: 0.0, gamma_6: 0.0, beta_7: 0.0, gamma_7: 0.0, tau_0: 0.0}  samples= None
E=-1.42574613  angles= {beta_0: 3.141592653589793, gamma_0: 0.0, beta_1: 3.141592653589793, gamma_1: 0.0, beta_2: 3.141592653589793, gamma_2: 0.0, beta_3: 3.141592653589793, gamma_3: 0.0, beta_4: 0.0, gamma_4: 0.0, beta_5: 0.0, gamma_5: 0.0, beta_6: 0.0, gamma_6: 0.0, beta_7: 0.0, gamma_7: 0.0, tau_0: 0.16144418716430664}  samples= None
E=-1.49236555  angles= {beta_0: 3.1415926535897

We see that the QCC energy converged to the FCI energy with only a single entangler! 

### LiH in 6-31G basis

Let us move on to the problem of LiH in 6-31G basis using the same active space as the UCCSD example.

In [11]:
xyz_data = get_molecular_data('liH', geometry=1, xyz_format=True)

basis = '6-31g'
active = {'B1':[0,1], 'A1':[2,3]}
lih = tq.quantumchemistry.Molecule(geometry=xyz_data, basis_set = basis, active_orbitals = active)
hf_reference = hf_occ(2*liH.n_orbitals, lih.n_electrons)


H = lih.make_hamiltonian()

print("\nHamiltonian has {} terms\n".format(len(H)))

#Define number of entanglers to enter ansatz
n_ents = 6

#Rank entanglers using energy gradient criterion
ranked_entangler_groupings = generate_QCC_gradient_groupings(H.to_openfermion(), 
                                                             2*lih.n_orbitals, 
                                                             hf_reference, 
                                                             cutoff=threshold)

print('Grouping gradient magnitudes (Grouping : Gradient magnitude):')
for i in range(len(ranked_entangler_groupings)):
    print('{} : {}'.format(i+1,ranked_entangler_groupings[i][1]))

entanglers = get_QCC_entanglers(ranked_entangler_groupings, n_ents, 2*lih.n_orbitals)

print('\nSelected entanglers:')
for ent in entanglers:
    print(ent)

ValueError: ('liH', 'Unknown moleucles given')

With the 6 chosen entanglers, we can then run the QCC VQE simulation.

In [15]:
#Mean-field part of U (Omega):    
U_MF = construct_QMF_ansatz(n_qubits = 2*lih.n_orbitals)
#Entangling part of U:
U_ENT = construct_QCC_ansatz(entanglers)

U_QCC = U_MF + U_ENT

E = tq.ExpectationValue(H=H, U=U_QCC)

initial_vals = init_qcc_params(hf_reference, E.extract_variables())

#Minimize wrt the entangler amplitude and MF angles:
result = tq.minimize(objective=E, method="BFGS", initial_values=initial_vals, tol=1.e-4)


print('\nObtained QCC energy ({} entanglers): {}'.format(len(entanglers), result.energy))

Optimizer: <class 'tequila.optimizers.optimizer_scipy.OptimizerSciPy'> 
backend         : qulacs
samples         : None
save_history    : True
noise           : None

Method          : BFGS
Objective       : 1 expectationvalues
gradient        : 44 expectationvalues

active variables : 22

E=-75.98071144  angles= {beta_0: 3.141592653589793, gamma_0: 0.0, beta_1: 3.141592653589793, gamma_1: 0.0, beta_2: 3.141592653589793, gamma_2: 0.0, beta_3: 3.141592653589793, gamma_3: 0.0, beta_4: 0.0, gamma_4: 0.0, beta_5: 0.0, gamma_5: 0.0, beta_6: 0.0, gamma_6: 0.0, beta_7: 0.0, gamma_7: 0.0, tau_0: 0.0, tau_1: 0.0, tau_2: 0.0, tau_3: 0.0, tau_4: 0.0, tau_5: 0.0}  samples= None
E=-75.98783364  angles= {beta_0: 3.141592653589793, gamma_0: 0.0, beta_1: 3.141592653589793, gamma_1: 0.0, beta_2: 3.141592653589793, gamma_2: 0.0, beta_3: 3.141592653589793, gamma_3: 0.0, beta_4: 0.0, gamma_4: 0.0, beta_5: 0.0, gamma_5: 0.0, beta_6: 0.0, gamma_6: 0.0, beta_7: 0.0, gamma_7: 0.0, tau_0: 0.07584381103515625, 

We obtain chemical accuracy for water near equilibrium geometry with only 6 entanglers. The obtained energy is not as accurate as that of UCCSD for this problem, however the QCC optimization may be performed at a fraction of the UCCSD circuit depth. One can also increase the number of entanglers entering the QCC ansatz to increase accuracy. As a final check, one can always run $n$ VQE trials with random initial guesses to test if the optimization fell into a local minimum. ***(Warning: Completing n=10 trials may take a few minutes for this VQE instance).***

In [16]:
n = 10
result = minimize_E_random_guesses(objective=E, method='BFGS', tol=1e-4, n=n)

print('\nObtained QCC energy ({} entanglers): {}'.format(len(entanglers), result))

Optimizer: <class 'tequila.optimizers.optimizer_scipy.OptimizerSciPy'> 
backend         : qulacs
samples         : None
save_history    : True
noise           : None

Method          : BFGS
Objective       : 1 expectationvalues
gradient        : 44 expectationvalues

active variables : 22

E=-73.97466678  angles= {beta_0: 2.5032792734674794, gamma_0: 4.269736231771848, beta_1: 1.99917246071, gamma_1: 0.4812818976026809, beta_2: 0.23634587162363235, gamma_2: 4.935591850318382, beta_3: 1.2698509301366092, gamma_3: 3.0956601125363052, beta_4: 3.8814849851352093, gamma_4: 2.1744926868038967, beta_5: 4.975297793180924, gamma_5: 5.719673420447425, beta_6: 1.5657465590426605, gamma_6: 3.253714588395857, beta_7: 1.3775047856676284, gamma_7: 5.789096284346924, tau_0: 2.14515460370191, tau_1: 6.243533876021702, tau_2: 6.049270159892919, tau_3: 3.4464573133474614, tau_4: 0.848930291363962, tau_5: 1.6527129905147593}  samples= None
E=-74.50085436  angles= {beta_0: 2.39631897683662, gamma_0: 4.2252